In [2]:
import numpy as np 
import pandas as pd 
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import KFold, RepeatedKFold, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor
from scipy import sparse
import warnings
import time
import sys
import os
import re
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',None)
pd.set_option('max_colwidth',100)

In [15]:
train = pd.read_csv('../datasets/jinnan/jinnan_round1_train_20181227.csv', encoding = 'gb18030')
test  = pd.read_csv('../datasets/jinnan/jinnan_round1_testA_20181227.csv', encoding = 'gb18030')

In [40]:
train[train.A2 == 125]

,样本id,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18,A19,A20,A21,A22,A23,A24,A25,A26,A27,A28,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11,B12,B13,B14,收率
60,sample_296,300,125.0,NaN,980,17:30:00,24.0,NaN,NaN,19:00:00,100,20:00:00,102,0.2,21:00:00,104.0,22:00:00,105.0,0.2,300,0:00-0:30,35.0,9.0,5.0,1:00:00,70,1:30:00,78,10:30-11:30,300.0,3.5,3.5,11:30-12:30,12:30:00,60,14:30:00,40.0,16:30-17:30,18:00-19:30,NaN,800.0,0.15,400,0.902
120,sample_842,300,125.0,NaN,980,14:30:00,24.0,NaN,NaN,16:00:00,100,17:00:00,102,0.2,18:00:00,102.0,19:00:00,103.0,0.2,300,19:10-19:40,50.0,10.0,5.0,20:00:00,70,20:30:00,78,1:00-2:00,320.0,3.5,3.5,2:00-3;00,3:00:00,60,6:00:00,40.0,9:00-10:30,11:30-13:00,NaN,800.0,0.15,280,0.624
142,sample_423,300,125.0,NaN,980,20:30:00,21.0,NaN,NaN,22:30:00,100,23:30:00,102,0.2,0:30:00,104.0,1:30:00,106.0,0.2,100,1:30-2:00,50.0,9.0,5.0,2:30:00,80,3:00:00,77,3:00-4:00,260.0,3.5,3.5,3:30-4:30,4:30:00,65,5:30:00,45.0,5:30-7:00,7:00-8:00,NaN,800.0,0.15,400,0.891
175,sample_432,300,125.0,NaN,980,12:30:00,21.0,NaN,NaN,14:30:00,100,15:30:00,102,0.2,16:30:00,104.0,17:30:00,106.0,0.2,100,17:30-18:00,50.0,9.0,5.0,18:30:00,80,19:00:00,78,19:00-20:00,290.0,3.5,3.5,20:00-21:00,21:00:00,65,23:00:00,45.0,23:00-0:00,0:30-1:30,NaN,800.0,0.15,400,0.902
244,sample_286,300,125.0,NaN,980,2:00:00,19.0,NaN,NaN,3:30:00,100,4:30:00,102,0.2,5:30:00,104.0,6:30:00,105.0,0.2,300,6:30-7:30,30.0,10.0,5.0,7:30:00,77,8:00:00,70,10:30-11:30,300.0,3.5,3.5,11:30-13:00,13:00:00,65,16:00:00,45.0,16:00-17:30,18:00-19:30,NaN,800.0,0.15,400,0.891
260,sample_427,300,125.0,NaN,980,6:00:00,21.0,NaN,NaN,7:30:00,100,9:00:00,102,0.2,10:00:00,104.0,11:00:00,106.0,0.2,100,11:30-12:00,50.0,9.0,5.0,12:30:00,80,13:00:00,78,13:00-14:00,300.0,3.5,3.5,14:00-15:00,15:00:00,65,16:30:00,45.0,16:30-17:30,18:00-19:00,NaN,800.0,0.15,400,0.902
300,sample_301,300,125.0,NaN,980,19:00:00,21.0,NaN,NaN,20:30:00,100,21:30:00,101,0.2,22:30:00,103.0,23:30:00,105.0,0.2,300,23:00-0:00,40.0,10.0,5.0,0:00:00,70,0:30:00,78,10:30-11:30,300.0,3.5,3.5,11:30-13:00,13:00:00,60,15:00:00,45.0,15:00-16:30,17:30-19:00,NaN,800.0,0.15,400,0.913
310,sample_284,300,125.0,NaN,980,14:00:00,21.0,NaN,NaN,15:30:00,100,16:30:00,102,0.2,17:30:00,104.0,18:30:00,106.0,0.2,300,19:00-19:30,40.0,10.0,5.0,19:30:00,80,20:00:00,75,20:00-21:00,280.0,3.5,3.5,21:00-22:30,22:30:00,60,23:30:00,50.0,23:30-1:00,1:30-3:00,NaN,800.0,0.15,400,0.902
364,sample_859,300,125.0,NaN,980,3:30:00,21.0,NaN,NaN,5:00:00,100,6:00:00,102,0.2,7:00:00,103.0,8:00:00,104.0,0.2,300,11;30-12:00,40.0,10.0,5.0,12:00:00,70,12:30:00,78,19:00-20:00,300.0,3.5,3.5,20:00-21:30,21:30:00,60,3:00:00,40.0,3;30-5:00,5:30-7:00,NaN,800.0,0.15,400,0.891
369,sample_845,300,125.0,NaN,980,15:00:00,21.0,NaN,NaN,16:00:00,100,17:30:00,101,0.2,18:30:00,103.0,19:30:00,105.0,0.2,300,22:00-22:30,40.0,10.0,5.0,22:40:00,70,23:00:00,78,6:00-7:00,300.0,3.5,3.5,7:00-8:30,8:30:00,60,11:00:00,40.0,12:30-14:00,14:30-16;00,NaN,800.0,0.15,400,0.891
